In [ ]:
def get_today_date():
    from datetime import datetime
    date_str = str(datetime.now())
    today=datetime.strptime(date_str, '%Y-%m-%d %H:%M:%S.%f')
    return today.strftime('%d-%b-%Y')

def get_option_chain(symbol):
    today = get_today_date()
    # filename_OI_Chng = symbol + '-CHNG-IN-OI-' + today + ".csv"
    # filename_OI_Total =  symbol + '-TOTAL-OI-' + today + ".csv"
    # filename_ce_option_chain =  symbol +'-CE-OPTION-CHAIN-' + today + ".csv"
    # filename_pe_option_chain =  symbol +'-PE-OPTION-CHAIN-' + today + ".csv"
    # filename_ce_full_option_chain =  symbol +'-CE-FULL-OPTION-CHAIN-' + today + ".csv"
    # filename_pe_full_option_chain =  symbol +'-PE-FULL-OPTION-CHAIN-' + today + ".csv"
    
    outdir = "D:\Jupyter Python\Stocks-office\json-csv\\"
    
    file_OI_Chng = symbol + '-CHNG-IN-OI-' + today + ".csv"
    file_OI_Total =  symbol + '-TOTAL-OI-' + today + ".csv"
    file_ce_option_chain =  symbol +'-CE-OPTION-CHAIN-' + today + ".csv"
    file_pe_option_chain =  symbol +'-PE-OPTION-CHAIN-' + today + ".csv"
    file_ce_full_option_chain =  symbol +'-CE-FULL-OPTION-CHAIN-' + today + ".csv"
    file_pe_full_option_chain =  symbol +'-PE-FULL-OPTION-CHAIN-' + today + ".csv"
    
    filename_OI_Chng = os.path.join(outdir, file_OI_Chng)
    filename_OI_Total =  os.path.join(outdir, file_OI_Total)
    filename_ce_option_chain =  os.path.join(outdir, file_ce_option_chain)
    filename_pe_option_chain =  os.path.join(outdir, file_pe_option_chain)
    filename_ce_full_option_chain =  os.path.join(outdir, file_ce_full_option_chain)
    filename_pe_full_option_chain =  os.path.join(outdir, file_pe_full_option_chain)
    
   
    hd =pd.DataFrame({'Time':[],'CE_OI_CHNG':[],'PE_OI_CHNG':[]})
    hd.to_csv(filename_OI_Chng,header=True,index=False)
    
    hd2 =pd.DataFrame({'Time':[],'CE_OI_TOTAL':[],'PE_OI_TOTAL':[]})
    hd2.to_csv(filename_OI_Total,header=True,index=False)
   
    url='https://www.nseindia.com/api/option-chain-indices?symbol='+ symbol
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:104.0) Gecko/20100101 Firefox/104.0'}
    starttime=time.time()
    timeout = time.time() + 60*420  # 60 seconds times 2 meaning the script will run for 2 minutes
    while time.time() <= timeout:
        try:
            r = requests.get(url,headers=headers)
            data=r.json()
           
            list_ce=[]
            list_pe=[]
            list_time=[]
            j=data["filtered"]["data"]
            dict_={'strikePrice': 0,                  
                 'expiryDate': '',
                 'underlying': 'NIFTY',
                 'identifier': '',
                 'openInterest': 0,
                 'changeinOpenInterest': 0,
                 'pchangeinOpenInterest': 0,
                 'totalTradedVolume': 0,
                 'impliedVolatility': 0,
                 'lastPrice': 0,
                 'change': 0,
                 'pChange': 0,
                 'totalBuyQuantity': 0,
                 'totalSellQuantity': 0,
                 'bidQty': 0,
                 'bidprice': 0,
                 'askQty': 0,
                 'askPrice': 0,
                 'underlyingValue':0}

            for i in range(len(j)):
                list_time.append(data["records"]["timestamp"])
                try:
                    list_ce.append(data["filtered"]["data"][i]["CE"])
                except:
                    list_ce.append(dict_)
                try:
                    list_pe.append(data["filtered"]["data"][i]["PE"])
                except :
                    list_pe.append(dict_)

            df_ce=pd.DataFrame(list_ce)
            df_pe=pd.DataFrame(list_pe)
            df_time=pd.DataFrame(list_time,columns=["timestamp"])
            df_ce["timestamp"]=df_time["timestamp"]
            df_pe["timestamp"]=df_time["timestamp"]


            df_ce_reorder = df_ce[['timestamp','strikePrice', 'expiryDate', 'underlying', 'openInterest','changeinOpenInterest', 
                            'pchangeinOpenInterest','impliedVolatility', 'lastPrice', 'change', 'pChange', 'underlyingValue']] 
            df_ce_reorder.columns=['timestamp','strike', 'expiry', 'index', 'oi','oi_change', 
                            'oi%','iv', 'ltp', 'ltp_change', 'ltp%', 'underlying']
            df_pe_reorder = df_pe[['timestamp','strikePrice', 'expiryDate', 'underlying', 'openInterest','changeinOpenInterest', 
                            'pchangeinOpenInterest','impliedVolatility', 'lastPrice', 'change', 'pChange', 'underlyingValue']] 
            df_pe_reorder.columns=['timestamp','strike', 'expiry', 'index', 'oi','oi_change', 
                            'oi%','iv', 'ltp', 'ltp_change', 'ltp%', 'underlying']
            ce_oi_change=df_ce_reorder["oi_change"].sum()
            pe_oi_change=df_pe_reorder["oi_change"].sum()
            ce_oi_total=df_ce_reorder["oi"].sum()
            pe_oi_total=df_pe_reorder["oi"].sum()
            
            
            time1 = data["records"]["timestamp"]
            oi_change=pd.DataFrame({"Time":[time1],"CE_OI_CHNG":[ce_oi_change],"PE_OI_CHNG":[pe_oi_change]})
            oi_total=pd.DataFrame({"Time":[time1],"CE_OI_TOTAL":[ce_oi_total],"PE_OI_TOTAL":[pe_oi_total]})

            oi_change.to_csv(filename_OI_Chng,mode='a',header=False,index=False)
            oi_total.to_csv(filename_OI_Total,mode='a',header=False,index=False)

            df_ce_reorder.to_csv(filename_ce_option_chain,header=True,index=False)
            df_pe_reorder.to_csv(filename_pe_option_chain,header=True,index=False)
            
            df_ce_reorder.to_csv(filename_ce_full_option_chain,mode='a',header=False,index=False)
            df_pe_reorder.to_csv(filename_pe_full_option_chain,mode='a',header=False,index=False)

            time.sleep(180 - ((time.time() - starttime) % 180.0)) # 5 second interval between each new iteration
            # print(oi_change)
            # print(oi_total)
        except :
            print("Network error.....reconnecting...",starttime)
            
######################### Main program ######################

import pandas as pd
import requests
import os
import time
get_option_chain('NIFTY')


Network error.....reconnecting... 1665460102.933607
Network error.....reconnecting... 1665460102.933607
Network error.....reconnecting... 1665460102.933607
Network error.....reconnecting... 1665460102.933607
Network error.....reconnecting... 1665460102.933607
Network error.....reconnecting... 1665460102.933607
Network error.....reconnecting... 1665460102.933607
Network error.....reconnecting... 1665460102.933607
Network error.....reconnecting... 1665460102.933607
Network error.....reconnecting... 1665460102.933607
Network error.....reconnecting... 1665460102.933607
Network error.....reconnecting... 1665460102.933607
Network error.....reconnecting... 1665460102.933607
Network error.....reconnecting... 1665460102.933607
Network error.....reconnecting... 1665460102.933607
Network error.....reconnecting... 1665460102.933607
Network error.....reconnecting... 1665460102.933607
Network error.....reconnecting... 1665460102.933607
Network error.....reconnecting... 1665460102.933607
Network erro

In [7]:
import pandas as pd
import requests
import time
symbol='NIFTY'
url='https://www.nseindia.com/api/option-chain-indices?symbol='+ symbol
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:104.0) Gecko/20100101 Firefox/104.0'}
    # starttime=time.time()
    # timeout = time.time() + 60*420  # 60 seconds times 2 meaning the script will run for 2 minutes
# while time.time() <= timeout:
    # try:
r = requests.get(url,headers=headers)
data=r.json()

list_ce=[]
list_pe=[]
list_time=[]
j=data["filtered"]["data"]

dict_={'strikePrice': 0,
 'expiryDate': '',
 'underlying': 'NIFTY',
 'identifier': '',
 'openInterest': 0,
 'changeinOpenInterest': 0,
 'pchangeinOpenInterest': 0,
 'totalTradedVolume': 0,
 'impliedVolatility': 0,
 'lastPrice': 0,
 'change': 0,
 'pChange': 0,
 'totalBuyQuantity': 0,
 'totalSellQuantity': 0,
 'bidQty': 0,
 'bidprice': 0,
 'askQty': 0,
 'askPrice': 0,
 'underlyingValue':0}

for i in range(len(j)):
    list_time.append(data["records"]["timestamp"])
    try:
        list_ce.append(data["filtered"]["data"][i]["CE"])
    except:
        list_ce.append(dict_)
    try:
        list_pe.append(data["filtered"]["data"][i]["PE"])
    except :
        list_pe.append(dict_)

df_ce=pd.DataFrame(list_ce)
df_pe=pd.DataFrame(list_pe)
df_time=pd.DataFrame(list_time,columns=["timestamp"])
df_ce["timestamp"]=df_time["timestamp"]
df_pe["timestamp"]=df_time["timestamp"]


df_ce_reorder = df_ce[['timestamp','strikePrice', 'expiryDate', 'underlying', 'openInterest','changeinOpenInterest', 
                'pchangeinOpenInterest','impliedVolatility', 'lastPrice', 'change', 'pChange', 'underlyingValue']] 
df_ce_reorder.columns=['timestamp','strike', 'expiry', 'index', 'oi','oi_change', 
                'oi%','iv', 'ltp', 'ltp_change', 'ltp%', 'underlying']
df_pe_reorder = df_pe[['timestamp','strikePrice', 'expiryDate', 'underlying', 'openInterest','changeinOpenInterest', 
                'pchangeinOpenInterest','impliedVolatility', 'lastPrice', 'change', 'pChange', 'underlyingValue']] 
df_pe_reorder.columns=['timestamp','strike', 'expiry', 'index', 'oi','oi_change', 
                'oi%','iv', 'ltp', 'ltp_change', 'ltp%', 'underlying']
ce_oi_change=df_ce_reorder["oi_change"].sum()
pe_oi_change=df_pe_reorder["oi_change"].sum()
ce_oi_total=df_ce_reorder["oi"].sum()
pe_oi_total=df_pe_reorder["oi"].sum()


time1 = data["records"]["timestamp"]
oi_change=pd.DataFrame({"Time":[time1],"CE_OI_CHNG":[ce_oi_change],"PE_OI_CHNG":[pe_oi_change]})
oi_total=pd.DataFrame({"Time":[time1],"CE_OI_TOTAL":[ce_oi_total],"PE_OI_TOTAL":[pe_oi_total]})


In [9]:
df_pe

,strikePrice,expiryDate,underlying,identifier,openInterest,changeinOpenInterest,pchangeinOpenInterest,totalTradedVolume,impliedVolatility,lastPrice,change,pChange,totalBuyQuantity,totalSellQuantity,bidQty,bidprice,askQty,askPrice,underlyingValue,timestamp
0,10000,29-Sep-2022,NIFTY,OPTIDXNIFTY29-09-2022PE10000.00,2015,-39,-1.898734,302,141.15,0.65,-0.10,-13.333333,42650,28800,50,0.65,1300,0.70,17476.7,23-Sep-2022 09:44:48
1,11000,29-Sep-2022,NIFTY,OPTIDXNIFTY29-09-2022PE11000.00,2074,4,0.193237,44,120.57,0.80,0.10,14.285714,11450,9050,200,0.70,1600,0.85,17476.7,23-Sep-2022 09:44:48
2,12000,29-Sep-2022,NIFTY,OPTIDXNIFTY29-09-2022PE12000.00,2168,0,0.000000,104,97.61,0.65,-0.10,-13.333333,13800,5600,2800,0.65,650,0.80,17476.7,23-Sep-2022 09:44:48
3,12750,29-Sep-2022,NIFTY,OPTIDXNIFTY29-09-2022PE12750.00,0,0,0.000000,0,0.00,0.00,0.00,0.000000,50,0,50,0.05,0,0.00,17476.7,23-Sep-2022 09:44:48
4,13000,29-Sep-2022,NIFTY,OPTIDXNIFTY29-09-2022PE13000.00,3655,75,2.094972,147,79.71,0.80,0.05,6.666667,84650,6150,1200,0.70,1150,0.90,17476.7,23-Sep-2022 09:44:48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
118,19550,29-Sep-2022,NIFTY,OPTIDXNIFTY29-09-2022PE19550.00,0,0,0.000000,0,0.00,0.00,0.00,0.000000,9100,11100,1500,1965.05,1500,2143.50,17476.7,23-Sep-2022 09:44:48
119,20000,29-Sep-2022,NIFTY,OPTIDXNIFTY29-09-2022PE20000.00,16034,-11,-0.068557,14,63.48,2520.50,170.30,7.246192,16450,11450,250,2514.05,100,2524.65,17476.7,23-Sep-2022 09:44:48
120,21000,29-Sep-2022,NIFTY,OPTIDXNIFTY29-09-2022PE21000.00,508,0,0.000000,0,0.00,0.00,0.00,0.000000,12100,9300,50,3511.35,50,3524.15,17476.7,23-Sep-2022 09:44:48
121,22000,29-Sep-2022,NIFTY,OPTIDXNIFTY29-09-2022PE22000.00,32,0,0.000000,0,0.00,0.00,0.00,0.000000,10250,8750,250,4504.50,50,4528.90,17476.7,23-Sep-2022 09:44:48


In [4]:
list_ce[0]

{'strikePrice': 10000,
 'expiryDate': '29-Sep-2022',
 'underlying': 'NIFTY',
 'identifier': 'OPTIDXNIFTY29-09-2022CE10000.00',
 'openInterest': 3,
 'changeinOpenInterest': 0,
 'pchangeinOpenInterest': 0,
 'totalTradedVolume': 0,
 'impliedVolatility': 0,
 'lastPrice': 0,
 'change': 0,
 'pChange': 0,
 'totalBuyQuantity': 3850,
 'totalSellQuantity': 11750,
 'bidQty': 250,
 'bidprice': 7431.25,
 'askQty': 300,
 'askPrice': 7532.75,
 'underlyingValue': 17492.5}

In [5]:
dict={'strikePrice': 0,
 'expiryDate': '29-Sep-2022',
 'underlying': 'NIFTY',
 'identifier': 0,
 'openInterest': 0,
 'changeinOpenInterest': 0,
 'pchangeinOpenInterest': 0,
 'totalTradedVolume': 0,
 'impliedVolatility': 0,
 'lastPrice': 0,
 'change': 0,
 'pChange': 0,
 'totalBuyQuantity': 0,
 'totalSellQuantity': 0,
 'bidQty': 0,
 'bidprice': 0,
 'askQty': 0,
 'askPrice': 0,
 'underlyingValue': 17492.5}

In [6]:
dict

{'strikePrice': 0,
 'expiryDate': '29-Sep-2022',
 'underlying': 'NIFTY',
 'identifier': 0,
 'openInterest': 0,
 'changeinOpenInterest': 0,
 'pchangeinOpenInterest': 0,
 'totalTradedVolume': 0,
 'impliedVolatility': 0,
 'lastPrice': 0,
 'change': 0,
 'pChange': 0,
 'totalBuyQuantity': 0,
 'totalSellQuantity': 0,
 'bidQty': 0,
 'bidprice': 0,
 'askQty': 0,
 'askPrice': 0,
 'underlyingValue': 17492.5}